In [ ]:
import sqlite3
import pandas as pd

# Create a connection to the SQLite database
conn = sqlite3.connect('police_data.db')

#Create a dataframe for stop and search
stop_and_search_df = pd.read_sql_query("SELECT * from stop_and_search", conn)

# Close the connection
conn.close()

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


# Drop NaN values of latitude and longitude
stop_and_search_df.dropna(subset=['latitude', 'longitude'], inplace=True)

# Load borough boundaries
borough_boundaries = gpd.read_file("london_boroughs.geojson")

# Exclude City of London as we dont need it
borough_boundaries = borough_boundaries[borough_boundaries['name'] != 'City of London']

# Create Points from latitude and longitude for each row.
stop_and_search_df['Point'] = stop_and_search_df.apply(lambda x: Point(x['longitude'], x['latitude']), axis=1)

# Create Geometries from the points
points_gdf = gpd.GeoDataFrame({'geometry': stop_and_search_df['Point']}, crs="EPSG:4326")

# Create borough column
stop_and_search_df["Borough"] = None

# Assign borough to each point
for i in range(len(borough_boundaries)):
    mask = points_gdf.within(borough_boundaries.iloc[i]['geometry'])
    stop_and_search_df.loc[mask, "Borough"] = borough_boundaries.iloc[i]['name']

# Drop 'Point' column
stop_and_search_df.drop('Point', axis=1, inplace=True)

# Drop rows without a borough
stop_and_search_df.dropna(subset=['Borough'], inplace=True)
